<a href="https://colab.research.google.com/github/syeong1218/RL/blob/master/15_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A2C, A3C, PPO의 알고리즘을 비교해보았습니다.

## A2C 알고리즘



- 액터-크리틱 알고리즘

  : N개의 샘플->정책 업데이트->반복
  1. 크리틱과 액터 신경망의 파라미터 $\phi,\theta$ 초기화
  2.  반복
    1. N시간스텝 동안 아래의 과정 반복
      1. 정책으로 행동을 확률적으로 선택
      2. 행동의 보상과 다음 상태변수를 측정
      3. 샘플 저장
    2. 시간차 타깃 계산
    3. 크리틱 신경망의 손실함수 계산 

    $L=\frac{1}{2}\sum_{i}^{}(y_{i}-V_{\phi }(x_{i}))^{2}$
    4. 어드밴티지 계산
    
    $A_{\phi}(x_{i},u_{i})= r(x_{i},u_{i})+\gamma V_{\phi }(x_{i+1})-V_{\phi }(x_{i}), i=1,\cdots ,N$

    5. 크리틱 신경망 업데이트

    $\phi\leftarrow \phi +\alpha_{critic} \sum_{i}^{}\left [ (y_{i}-V_\phi (x_{i}))\triangledown _{\phi }V_{\phi }(x_i) \right ]$

    6. 액터 신경망 업데이트

    $\theta \leftarrow \theta+\alpha_{actor} \triangledown _{\theta }\sum_{i}^{}(log\pi _{\theta }(u_{i}|x_{i})A_{\phi }(x_{i},u_{i}))$


- 온라인 액터-크리틱 알고리즘
  : 한 개의 샘플->신경망 업데이트-> 반복
  1. 크리틱과 액터 신경망의 파라미터 $\phi,\theta$ 초기화
  2. 반복
    1. 정책으로 행동을 확률적으로 선택
    2. 행동의 보상과 다음 상태변수 측정
    3. 시간차 오차 계산
    
    $\rho _{t}= r(x_{t},u_{t})+\gamma V_{\phi }(x_{t+1})-V_{\phi }(x_{t})$

    4. 크리틱 신경망 업데이트
    
    $\phi\leftarrow \phi +\alpha_{critic}\rho _{t} \triangledown _{\phi }V_{\phi }(x_{t}) $

    5. 액터 신경망 업데이트

    $\theta \leftarrow \theta+\alpha_{actor} \triangledown _{\theta }(\rho_tlog\pi _{\theta }(u_{t}|x_{t}))$




## A3C 알고리즘

<br>

##### 1. 글로벌 크리틱과 액터 신경망의 파라미터 $\phi$와 $\theta$ 초기화
##### 2. 다중 에이전트(워커) 생성 후 워커별로 $\phi_w$와 $\theta_w$를 글로벌 신경망에 동기화
##### 3. 워커별로 정책으로 행동을 선택, 행동으로부터 상태변수 측정, 샘플 저장
##### 4. 워커의 n-스탭 시간차 타깃 계산 후 어드밴티지 계산
##### 5. 워커 크리틱 신경망 그래디언트 계산
$$\sum_{i=1}[(y_{w,i}-V_{\phi_w}(x_i)\nabla_{\phi_w}V_{\phi_w}(x_i)]$$
##### 6. 워커 액터 신경망 그래디언트 계산
$$\nabla_{\theta_w}\sum_i(log\pi_{\theta_w}(u_t \mid x_t)A_{\phi_w}(x_t, u_t)+\beta H(\pi_{\theta_w}(x_t \mid u_t)))$$
##### 7. 글로벌 신경망으로 워커의 그래디언트 송부
##### 8. 워커의 그래디언트로 글로벌 신경망 업데이트
$$\phi \leftarrow \phi + \alpha_{critic}\sum_{i=1}[(y_{w,i}-V_{\phi_w}(x_i)\nabla_{\phi_w}V_{\phi_w}(x_i)]$$
$$\theta \leftarrow \theta + \alpha_{actor}\nabla_{\theta_w}\sum_i(log\pi_{\theta_w}(u_t \mid x_t)A_{\phi_w}(x_t, u_t)+\beta H(\pi_{\theta_w}(x_t \mid u_t)))$$ 
##### 9. 업데이트된 글로벌 신경망 파라미터를 워커로 복사

<br>

#### 데이터 병렬화 A3C의 알고리즘은 위의 과정에서 워커가 샘플을 글로벌 신경망으로 전달하고, 글로벌 신경망은 그 샘플을 통해 업데이트되는 과정으로 진행된다.

## PPO 알고리즘

#### 1. 크리틱과 액터 신경망의 파라미터 $\phi$, $\theta$, 를 초기화한다.
#### 2. 반복

1)  N 시간 스텝 동안 반복

``` 
      (1) 이전 정책을 가우시안으로 가정하고 평균과 표준편차를 계산
      (2) 이전정책으로 부터 확률적으로 행동을 선택
      (3) 이전 정책의 로그-확률밀도함수 계산
```
$$ log\pi _{\theta }(u_{i}|x_{i})=-\sum_{j=1}^{n}\left [ \frac{(u_{i,j}-\mu _{\theta ,j}(x_{i}))^{2}}{2\sigma _{\theta ,j}^{2}(x_{i})}+\frac{1}{2}log(2\pi\sigma _{\theta ,j}^{2}(x_{i}) ) \right ]$$
```
      (4) 행동을 실행해 보상과 다음 상태변수를 측정
      (5) TD 타깃과 어드밴티지 계산
      (6) 데이터 모음를 배치에 저장
```

  2) 설정 에포크만큼 반복

 ```
      (1) 배치에서 설정된 미니배치 크기만큼 데이터를 추출
      (2) 크리틱 신경망의 손실함수를 계산
```
$$L= {1\over{2B}} \sum_i (y_i-V_\phi(x_i))^2$$
```
      (3) 크리틱 신경망 업데이트
```
$$\phi \leftarrow \phi + \beta \sum_i[(y_p-V_\phi(x_i))\nabla_\phi V_\phi(x_i)]$$
```
      (4) 이전 정책과 현재 정책의 비율 계산
```
$$r_t(\theta) = {{\pi_\theta(u_i|x_i)}\over{\pi_{\theta old}(u_i|x_i)}}$$
```
      (5) 대체 목적함수 그래디언트를 계산하고 액터 신경망을 업데이트
```
$$L_i^{clip} = min\{r_t(\theta)A^{\pi \theta old}(x_i,u_i), clip(r_t(\theta), 1-\epsilon, 1+\epsilon)A^{\pi \theta old}(x_i,u_i)\}$$
$$\nabla_\theta L^{clip}(\theta) \approx \nabla_\theta \sum_i L_i^{clip}(\theta)$$
$$\theta \leftarrow \theta + \alpha \nabla_\theta L^{clip}(\theta)$$